# `anchor_transmitting.ipynb`
If we think about "transmitting" from the location of the anchor nodes the received signals, they should all construct at the location of the target node. 

In [19]:
from WSN import *
import mutual_information as mtin
from scipy.signal import find_peaks
from color import *
import os
from tqdm import tqdm

In [20]:
mtin.solve_default_fn_equ()

In [35]:
import cv2

def animate_color_video(fname, x):
    # BW
    # y = 255 * (x-x.min()) / (x.max()-x.min())
    # BW inverted
    #y = 255 * ( 1 - (x-x.min()) / (x.max()-x.min()) )
    y = x.astype(np.uint8)
    nt, nx, ny, _ = x.shape
    print(f"nt = {nt:d}, nx = {nx:d}, ny = {ny:d}")
    # write video using opencv
    frate = 30
    out = cv2.VideoWriter(fname, \
                          cv2.VideoWriter_fourcc(*'mp4v'), \
                          frate, (nx,ny))
    for i in range(0,nt):
        print(f"i = {i:d}/{nt:d}\r", end="")
        # img = np.ones((nx, ny, 3), dtype=np.uint8)
        # for j in range(3): img[:,:] = y[i,::-1,:]
        img = y[i, ::-1, :]
        out.write(img)
    out.release()
    print("")

In [44]:
c = 1.6424885622140555
dt = 0.1
period = 11.6
wavelength = c * period
N = 10
wsn = WSN(100, N=N, D=142, std=0, c=c)
wsn.reset_nodes()
# nodes = np.array([
#     [55, 55],
#     [32, 31],
#     [84, 27],
# ])
nodes = wsn.nodes
sigs = np.array([
    mtin.default_fn_sol[400:, 0, int(node[1]), int(node[0])]
    for node in nodes
])

10


In [45]:
# Helper function, interp_signal(sig, dt, t)
def interp_signal(sig, dt, t):
    i = t / dt
    if i < 0 or i >= len(sig):
        raise ValueError(f"t is not within the range of sig, [0, {len(sig) * dt}). Given {t}.")
    ii = int(i)
    if ii == 0 or ii == len(sig) - 1:
        return sig[ii]
    less = sig[ii]
    more = sig[ii + 1]
    return interpolate(less, more, i - ii)


In [46]:
node_color = [0, 0, 0xff]

In [47]:
frate = 30
num_frames = int(period * frate)
n = mtin.default_fn_equ_params["n"]
frames = np.empty((num_frames, n, n, 3))

for i in tqdm(range(num_frames)):
    phase_shift = i / frate
    frame = np.empty((n, n, 3))
    major_data = np.zeros((n, n))
    for node, sig in zip(nodes, sigs):
        # data = np.empty((n, n))
        # for x in range(n):
        #     for y in range(n):
        #         point = np.array([x, y])

        #         d = dist(point, node)
        #         t = d / c + phase_shift
        #         val = interp_signal(sig, dt, t)
        #         data[y, x] = val
        data = np.array([
            [
                interp_signal(sig, dt, dist(np.array([x, y]), node) / c + phase_shift)
                for x in range(n)
            ]
            for y in range(n)
        ])

        major_data += data

    for j in range(3): frame[:, :, j] = major_data
    frames[i] = frame




100%|██████████| 348/348 [06:29<00:00,  1.12s/it]


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [48]:

frames = 255 * (frames-frames.min()) / (frames.max()-frames.min())
frames = frames.astype(np.int8)
for node in nodes:
    frames[:, int(node[1]), int(node[0])] = node_color

C:\Users\brian\AppData\Local\Temp\ipykernel_27824\886943957.py:1: RuntimeWarning: overflow encountered in byte_scalars
  frames = 255 * (frames-frames.min()) / (frames.max()-frames.min())


In [49]:

dir = "./fn_videos/anchor_transmitting1/"
os.makedirs(dir, exist_ok=True)
mtin.save_params(mtin.default_fn_equ_params, dir + "params.json")
animate_color_video(dir + "video.mp4", frames)

nt = 348, nx = 100, ny = 100
i = 347/348
